In [1]:
import os
import glob
import numpy as np
import pandas as pd

In [2]:
relative_dir = "./experiment_id_0"
out_file = "table.csv"
out_rounded_file = "table_rounded.csv"

In [3]:
results_per_seed = glob.glob(os.path.join(relative_dir, "*.csv"))

In [4]:
dfs = []
for seed, result in enumerate(results_per_seed, 1):
    df = pd.read_csv(result)
    df["seed"] = seed
    dfs.append(df)

In [5]:
dfs = pd.concat(dfs)

In [6]:
results = dfs.groupby(["Learner", "Method"]).agg(["mean", "std"])["PEHE"]

In [7]:
results2 = results.copy().reset_index()

In [8]:
results2 = results2.pivot(index="Method", columns="Learner", values=["mean", "std"])

In [9]:
final = pd.DataFrame(
    index=["target", "shared_source_target", "radial_gan", "htce"],
    columns=pd.MultiIndex.from_tuples([
        ("SLearner", "mean"), ("SLearner", "std"),
        ("TLearner", "mean"), ("TLearner", "std"), 
        ("DRLearner", "mean"), ("DRLearner", "std"), 
        ("TARNet", "mean"), ("TARNet", "std"), 
    ]),
    data=np.nan
)

In [10]:
for method, r in final.iterrows():
    for learner, value in r.keys():
        final.loc[method, (learner, value)] = results2.loc[method, (value, learner)]  # type: ignore

In [11]:
display(final)

SLearner            TLearner           DRLearner  \
                          mean       std      mean       std      mean   
target                0.331720  0.135690  0.244929  0.069735  0.189652   
shared_source_target  0.471942  0.331601  0.470483  0.312629  0.451806   
radial_gan            0.209571  0.078339  0.183383  0.043781  0.156344   
htce                  0.097702  0.030330  0.056987  0.023566  0.047714   

                                  TARNet            
                           std      mean       std  
target                0.094530  0.197481  0.097912  
shared_source_target  0.314064  0.464372  0.318526  
radial_gan            0.071768  0.177175  0.062531  
htce                  0.024043  0.079990  0.033935

In [12]:
final_rounded = final.round(decimals=3)
display(final_rounded)

SLearner        TLearner        DRLearner        TARNet  \
                         mean    std     mean    std      mean    std   mean   
target                  0.332  0.136    0.245  0.070     0.190  0.095  0.197   
shared_source_target    0.472  0.332    0.470  0.313     0.452  0.314  0.464   
radial_gan              0.210  0.078    0.183  0.044     0.156  0.072  0.177   
htce                    0.098  0.030    0.057  0.024     0.048  0.024  0.080   

                             
                        std  
target                0.098  
shared_source_target  0.319  
radial_gan            0.063  
htce                  0.034

In [13]:
final.to_csv(out_file)
final_rounded.to_csv(out_rounded_file)